In [1393]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
# from keras.layers import Dense, Dropout, Activation, LSTM, Convolution1D, MaxPooling1D, Flatten
# from keras.models import Sequential
import matplotlib.pyplot as plt

%matplotlib inline
from torchbearer import Trial
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1394]:
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision.datasets import MNIST
from torchbearer import Trial
import torchvision.transforms as transforms

In [1395]:
%ls

 驱动器 D 中的卷是 Data
 卷的序列号是 F66F-4A2F

 D:\projects\Stock-Market-Price-Prediction-master 的目录

2019/08/08  16:16    <DIR>          .
2019/08/08  16:16    <DIR>          ..
2019/08/08  16:13    <DIR>          .git
2019/08/08  14:53    <DIR>          .ipynb_checkpoints
2019/06/22  15:43           363,722 ^DJI.csv
2019/06/26  00:18           103,280 ^GSPC.csv
2019/07/11  16:40           165,190 ^GSPC_2011_06_01-2019_06_01.csv
2019/07/11  13:20    <DIR>          __pycache__
2019/07/20  13:26            41,117 1d_conv.weights
2019/07/20  08:46           381,476 5_1_CNN-Copy1.ipynb
2019/06/24  16:48           335,219 Archive.zip
2019/08/01  03:32               658 CBOE yearly 04-18.csv
2019/07/22  04:20    <DIR>          data
2019/08/01  03:32           189,414 EUR_GBP 05-16.csv
2019/08/01  03:32           189,007 EUR_JPY 05-16.csv
2019/08/01  03:32           189,396 EUR_USD 05-16.csv
2019/08/01  03:32           189,370 GBP_JPY 05-16.csv
2019/08/01  03:32           189,425 GBP_USD 05-16.csv
2019

In [1396]:
datapath = os.path.join(os.getcwd(), 'data')
datapath

'D:\\projects\\Stock-Market-Price-Prediction-master\\data'

In [1397]:
%ls data

 驱动器 D 中的卷是 Data
 卷的序列号是 F66F-4A2F

 D:\projects\Stock-Market-Price-Prediction-master\data 的目录

2019/07/22  04:20    <DIR>          .
2019/07/22  04:20    <DIR>          ..
2019/07/22  04:20           178,424 ^TNX.csv
2019/07/14  16:21           190,344 ^VIX.csv
2019/06/14  11:01               658 CBOE yearly 04-18.csv
2019/06/14  11:51           189,414 EUR_GBP 05-16.csv
2019/06/14  11:47           189,007 EUR_JPY 05-16.csv
2019/06/14  11:44           189,396 EUR_USD 05-16.csv
2019/06/14  11:49           189,370 GBP_JPY 05-16.csv
2019/06/14  11:54           189,425 GBP_USD 05-16.csv
2019/06/14  11:31           206,054 SP500 05-16.csv
               9 个文件      1,522,092 字节
               2 个目录 32,500,187,136 可用字节


# get the data

In [1398]:
data = []
for i, file in enumerate(os.listdir(datapath)):
    data.append(pd.read_csv(os.path.join(datapath,file))) 
for i in range(len(data)):
    print(os.listdir(datapath)[i], '\n', data[i].tail())

CBOE yearly 04-18.csv 
    Change Date     Rate (%)
32   13-Dec-17  1.25 - 1.50
33   21-Mar-18  1.50 - 1.75
34   13-Jun-18  1.75 - 2.00
35   26-Sep-18  2.00 - 2.25
36   19-Dec-18  2.25 - 2.50
EUR_GBP 05-16.csv 
               Date   Price    Open    High     Low Change %
3125  Jan 07, 2005  0.6981  0.7022  0.7049  0.6971   -0.64%
3126  Jan 06, 2005  0.7026  0.7041  0.7055  0.7016   -0.24%
3127  Jan 05, 2005  0.7043  0.7049  0.7073  0.7021   -0.17%
3128  Jan 04, 2005  0.7055  0.7069  0.7076  0.7044   -0.20%
3129  Jan 03, 2005  0.7069  0.7058  0.7097  0.7038   -0.01%
EUR_JPY 05-16.csv 
               Date   Price    Open    High     Low Change %
3125  Jan 07, 2005  136.80  138.23  138.74  136.44   -1.16%
3126  Jan 06, 2005  138.41  138.03  138.69  137.85    0.28%
3127  Jan 05, 2005  138.03  138.85  139.02  137.68   -0.58%
3128  Jan 04, 2005  138.83  138.34  139.05  137.77    0.38%
3129  Jan 03, 2005  138.31  139.07  139.31  138.06   -0.40%
EUR_USD 05-16.csv 
               Date   Price  

In [1399]:
#TNX: COBE 10 years interest rate
#VIX: COBE volitility index
for i in range(len(data)):
    print(len(data[i]))

37
3130
3130
3130
3130
3130
3021
3021
3021


In [1400]:
# take TNX, VIX and SP500 and prepocessing
sp500 = data[-3]
tnx = data[-2]
vix = data[-1]

#take only some columns
sp500= sp500.values[:, 1:5]
print (sp500.shape)
tnx= tnx.values[:, 1:5]
print (tnx.shape)
vix= vix.values[:, 1:5]
print (vix.shape)

(3021, 4)
(3021, 4)
(3021, 4)


In [1401]:
# split sample X and target y
X = np.zeros((3, 3020, 4))
X[0,:,:] = sp500[:3020, :]
X[1,:,:] = tnx[:3020, :]
X[2,:,:] = vix[:3020, :]

y = np.zeros((3,3020))
y[0,:] = sp500[1:, 0] #price
y[1,:] = tnx[1:, 3]
y[2,:] = vix[1:, 3]

print(X.shape, y.shape)

(3, 3020, 4) (3, 3020)


In [1402]:
print(sp500[:3020, :], sp500[1:, 0])

[[1202.079956 1211.920044 1217.800049 1200.319946]
 [1188.050049 1202.079956 1205.839966 1185.390015]
 [1183.73999 1188.050049 1192.72998 1183.719971]
 ...
 [2268.8798829999996 2266.22998 2273.820068 2266.149902]
 [2249.919922 2270.22998 2271.310059 2249.110107]
 [2249.26001 2249.5 2254.51001 2244.560059]] [1188.050049 1183.73999 1187.890015 ... 2249.919922 2249.26001 2238.830078]


In [1403]:
temp1 = X[0, :, 0]
temp2 = X[1, :, 3]
temp3 = X[2, :, 3]
X_temp = np.array([temp1, temp2, temp3]).T
X = X_temp

y = y[0,:]
print(X.shape, y.shape)

(3020, 3) (3020,)


In [1404]:
print(temp1, temp2, temp3)
print(X, y)

[1202.079956 1188.050049 1183.73999  ... 2268.879883 2249.919922
 2249.26001 ] [4.22  4.283 4.277 ... 2.563 2.506 2.477] [14.08 13.98 14.09 ... 11.99 12.95 13.37]
[[1202.079956    4.22       14.08    ]
 [1188.050049    4.283      13.98    ]
 [1183.73999     4.277      14.09    ]
 ...
 [2268.879883    2.563      11.99    ]
 [2249.919922    2.506      12.95    ]
 [2249.26001     2.477      13.37    ]] [1188.050049 1183.73999  1187.890015 ... 2249.919922 2249.26001
 2238.830078]


In [1405]:
test_dict = {'sp500': X[:,0], 'tnx':X[:,1], 'vix':X[:,2], 'y':y}
test_dict_df = pd.DataFrame(data=test_dict)

test_dict_df.tail()

,sp500,tnx,vix,y
3015,2260.959961,2.553,11.43,2263.790039
3016,2263.790039,2.543,11.44,2268.879883
3017,2268.879883,2.563,11.99,2249.919922
3018,2249.919922,2.506,12.95,2249.260010
3019,2249.260010,2.477,13.37,2238.830078


In [1406]:
test_dict_df= test_dict_df.dropna(axis=0)#drop rows which may contain missing values
test_dict_df.tail()

,sp500,tnx,vix,y
3015,2260.959961,2.553,11.43,2263.790039
3016,2263.790039,2.543,11.44,2268.879883
3017,2268.879883,2.563,11.99,2249.919922
3018,2249.919922,2.506,12.95,2249.260010
3019,2249.260010,2.477,13.37,2238.830078


# building the binary target

In [1407]:
# -1 --> tomorrow goes down 
# 0 --> tomorrow goes up

In [1408]:
# initialising
test_dict_df['forecasting'] = 0
test_dict_df.tail()


,sp500,tnx,vix,y,forecasting
3015,2260.959961,2.553,11.43,2263.790039,0
3016,2263.790039,2.543,11.44,2268.879883,0
3017,2268.879883,2.563,11.99,2249.919922,0
3018,2249.919922,2.506,12.95,2249.260010,0
3019,2249.260010,2.477,13.37,2238.830078,0


In [1409]:
df = test_dict_df
# if y > sp500: predict as go up
# else predict as go down

for i in df.index:
    if df['y'][i] > df['sp500'][i]:  # go up?
        df.at[i, 'forecasting'] = 1
    else:
        df.at[i, 'forecasting'] = 0
        

df.tail()

,sp500,tnx,vix,y,forecasting
3015,2260.959961,2.553,11.43,2263.790039,1
3016,2263.790039,2.543,11.44,2268.879883,1
3017,2268.879883,2.563,11.99,2249.919922,0
3018,2249.919922,2.506,12.95,2249.260010,0
3019,2249.260010,2.477,13.37,2238.830078,0


Now instead of set the next day's value as target,
we will set the forecasting as target

In [1410]:
X = df.values[:,0:3]
y = df.values[:,4] # set the forecasting column as target
print(X.shape, y.shape)

(3016, 3) (3016,)


# Normalisation over train and test set

no need normalising the target

In [1411]:
split_point = int(len(y) * 0.4)
y1 = y[:split_point]
y2 = y[split_point:]

X1 = X[:split_point,:]
X2 = X[split_point:,:]

# y1= (y1- min(y1))/(max(y1)- min(y1)) #??? normalsation
X1= (X1- np.min(X1,axis=0))/(np.max(X1, axis=0)-np.min(X1,axis=0))

# y2= (y2- min(y2))/(max(y2)- min(y2)) #??? normalsation
X2= (X2- np.min(X2,axis=0))/(np.max(X2, axis=0)-np.min(X2,axis=0))


X = np.concatenate((X1,X2) , axis = 0)
y = np.concatenate((y1,y2) , axis = 0)

print(X.shape, y.shape)

(3016, 3) (3016,)


In [1412]:
print(X,'\n', y)

[[0.59142258 0.67611846 0.05903903]
 [0.57563416 0.69596723 0.05762998]
 [0.57078387 0.69407687 0.05917993]
 ...
 [0.99772637 0.45547945 0.04432059]
 [0.98254795 0.43378995 0.0697983 ]
 [0.98201966 0.42275495 0.0809448 ]] 
 [0. 0. 1. ... 0. 0. 0.]


# seperating time step

In [1413]:
time_steps= 100 #1 to 100 days ahead
X_new= np.zeros((X.shape[0] - time_steps +1, time_steps, X.shape[1]))
y_new= np.zeros((y.shape[0] -time_steps +1,))
for ix in range(X_new.shape[0]):
    for jx in range(time_steps):
        X_new[ix, jx, :]= X[ix +jx, :]
    y_new[ix]= y[ix + time_steps -1]
print (X_new.shape, y_new.shape)

(2917, 100, 3) (2917,)


In [1414]:
y_new

array([1., 1., 0., ..., 0., 0., 0.])

# seperating train and test set

In [1415]:
split = split_point
X_train = X_new[:split]
X_test = X_new[split:]

y_train = y_new[:split]
y_test = y_new[split:]

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(1206, 100, 3) (1206,)
(1711, 100, 3) (1711,)


In [1416]:
lens = X_train.shape[0]
lens

1206

In [1417]:
#convert to torch
trainData = torch.from_numpy(X_train)
testData = torch.from_numpy(y_train)
validateData = torch.from_numpy(X_test)
print(trainData.shape, testData.shape, validateData.shape)
# testData = testData.view(testData.shape[0],3,-1)

torch.Size([1206, 100, 3]) torch.Size([1206]) torch.Size([1711, 100, 3])


In [1418]:
trainData = trainData.view(lens,-1,time_steps,3)
testData = testData.view(lens,-1)

validateData = validateData.view(validateData.shape[0],-1,time_steps,3)
print(trainData.shape,testData.shape,validateData.shape)

torch.Size([1206, 1, 100, 3]) torch.Size([1206, 1]) torch.Size([1711, 1, 100, 3])


# data loader

In [1419]:
#reload tensordata
train = TensorDataset(trainData.view(lens, -1).float(), testData.view(-1).float())

trainloader = DataLoader(train, batch_size=64, shuffle=True)

dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images)) 
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([64, 300])
torch.Size([64])


In [1420]:
print(trainData.view(lens, -1).shape, testData.view(-1).shape)
print(type(trainData))

torch.Size([1206, 300]) torch.Size([1206])
<class 'torch.Tensor'>


# create model and training

In [1421]:
# define baseline MLP model
class BaselineModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(BaselineModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)        
        out = self.fc2(out)
        out = F.sigmoid(out)
        
        return out
    

In [1422]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = BaselineModel(300, 500, 1)

# define the loss function and the optimiser
# loss_function = nn.MSELoss()

#classification problem  we use bce loss
loss_function = nn.BCELoss()
optimiser = optim.Adam(model.parameters())

trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=50)

[{'running_loss': 0.7203235030174255,
  'running_binary_acc': 0.5251687169075012,
  'loss': 0.7118006944656372,
  'binary_acc': 0.5063148736953735,
  'train_steps': 19,
  'validation_steps': None},
 {'running_loss': 0.7039076089859009,
  'running_binary_acc': 0.5230318307876587,
  'loss': 0.6890496015548706,
  'binary_acc': 0.5479620099067688,
  'train_steps': 19,
  'validation_steps': None},
 {'running_loss': 0.6978951692581177,
  'running_binary_acc': 0.5336009860038757,
  'loss': 0.6888723969459534,
  'binary_acc': 0.5461353659629822,
  'train_steps': 19,
  'validation_steps': None},
 {'running_loss': 0.6895086765289307,
  'running_binary_acc': 0.5380042195320129,
  'loss': 0.6886452436447144,
  'binary_acc': 0.5356714129447937,
  'train_steps': 19,
  'validation_steps': None},
 {'running_loss': 0.689754843711853,
  'running_binary_acc': 0.5302217602729797,
  'loss': 0.6940037608146667,
  'binary_acc': 0.5165961980819702,
  'train_steps': 19,
  'validation_steps': None},
 {'running_

In [1423]:
y_pred= model(validateData.view(validateData.shape[0],-1)[0].to(device).float())
y_pred

tensor([0.8240], device='cuda:0', grad_fn=<SigmoidBackward>)

## 

In [ ]:
validateData.shape
validateData.view(602,-1)[0]

In [ ]:
y_pred= model(validateData.view(602,-1).float())
y_pred.shape

plt.plot(y_pred.cpu().detach().numpy(), 'r-')
plt.plot(y_test, 'b-')
plt.show()

In [ ]:
print(trainData[0].shape,testData[0].shape)

# generate gradient -- need change here

In [ ]:
class gradient_generator():
    
    def __init__(self, model):
        self.model = model.to('cpu')
        self.gradients = None
        # Put model in evaluation mode
        self.model.eval()
        # Hook the first layer to get the gradient
        self.hook = self.hook_layers()
    def hook_layers(self):    
        def hook_function(module, grad_in, grad_out):
#             print("hook called")
#             print(grad_out[0])
            self.gradients = grad_out[0]
        first_layer = list(self.model._modules.items())[0][1]
        hook = first_layer.register_forward_hook(hook_function)
        return hook
    def generate_gradient(self, inputs):
        self.model.zero_grad()
        _ = model(inputs)
        self.hook.remove()
        return self.gradients
    

grad = torch.zeros((2412,9))
i = 0
for data in trainloader:
    # get the inputs
    inputs, labels = data
    
    GG = gradient_generator(model)
    temp = GG.generate_gradient(inputs)
    grad[i,:] = temp
    i += 1

grad